---
title: "Enumerating regular expressions"
author: "Martin Ong"
date: "2025-24-1"
format: 
  html:
    toc: true
    embed-resources: true
    code-fold: false
    code-tools: true
    code-line-numbers: true
    number-sections: true
---

In [ ]:
from __future__ import annotations
from collections import defaultdict, deque
from typing import Generic, Literal, Protocol, Self, TypeVar
from itertools import chain, combinations, product
from copy import deepcopy


EMPTYSET = "\u2205"
EPSILON = "\u03b5"

ALPHABET = "ab"
EXTENDED_ALPHABET = f"ab{EPSILON}"
T = TypeVar("T")
S = TypeVar("S")

# Regular expressions
In this post we will consider a recursive definition of regular expressions, as opposed
to the more general definition used in programming languages such as Perl.

We define a regular expression over an alphabet (a set of allowed characters)
$\Sigma$ as follows.

1. $\emptyset$ and $\varepsilon$ are regular expressions
2. $a$ is a regular expression for all $a$ in the alphabet
3. If $R_1, R_2$ are regular expressions, then $(R_1 | R_2)$ is a regular expression
4. If $R_1, R_2$ are regular expressions, then $(R_1 R_2)$ is a regular expression
5. If $R$ is a regular expression, then $R^{\*}$ is a regular expression

Semantically, we evaluate regular expressions as follows.
1. $\emptyset$ matches nothing, and $\varepsilon$ matches only an empty string
2. $a$ matches only the string containing the single character $a$
3. $(R_1 | R_2)$ matches any string that either is matched by $R_1$ or is matched by $R_2$
4. $(R_1 R_2)$ matches any string that can be split in two in such a way that the first
    segment is matched by $R_1$ and the second is matched by $R_2$
5. $R^{\*}$ matches any string that can be split into 0 or more segments which are
   all matched by $R$. Note that this includes the empty string

We will use the following classes to represent a regular expression.

In [ ]:
class Regex:
    name: str
    regex: list

    def __repr__(self) -> str:
        ...

class Empty(Regex):
    def __init__(self) -> None:
        self.regex = []
        self.name = "Empty"

    def __repr__(self) -> str:
        return "Empty()"

class Epsilon(Regex):
    def __init__(self) -> None:
        self.regex = []
        self.name = "Epsilon"

    def __repr__(self) -> str:
        return "Epsilon()"

class Char(Regex):
    def __init__(self, char: str) -> None:
        assert len(char) == 1
        assert char in ALPHABET
        self.char = char

    def __repr__(self) -> str:
        return f"Char({self.char})"

class Union(Regex):
    def __init__(self, r1: Regex, r2: Regex) -> None:
        self.r1 = r1
        self.r2 = r2
        self.regex = [self.r1, self.r2]
        self.name = "Union"

    def __repr__(self) -> str:
        return f"Union({self.r1, self.r2})"

class Concat(Regex):
    def __init__(self, r1: Regex, r2: Regex) -> None:
        self.r1 = r1
        self.r2 = r2
        self.regex = [self.r1, self.r2]
        self.name = "Concat"

    def __repr__(self) -> str:
        return f"Concat({self.r1, self.r2})"

class Star(Regex):
    def __init__(self, r: Regex) -> None:
        self.r = r

    def __repr__(self) -> str:
        return f"Star({self.r})"

# Enumerations
An infinite collection of objects can be enumerated by a process which when
run will repeatedly generate unique objects from the collection. According to this
definition of an enumeration, any object in the collection will eventually be generated
in a finite time.

In particular, the collection of all regular expressions is an infinite collection,
so naturally one might ask how we can enumerate all regular expressions.

## A trivial enumeration
One of the simplest collections to enumerate in computer science is the set of
all strings over an alphabet, often denoted $\Sigma^{*}$, where $\Sigma$ is 
the alphabet.

The set of all strings can be enumerated by first generating the empty string,
then all strings of length one, and then all strings of length two, and so on.

Clearly, any fixed string will have finite length, so eventually it will be generated 
by such a process.

### Adapting for regex 
Now, one might notice that all regular expressions are strings over the alphabet
$$\Sigma \cup \{ (, ), |, *, \varepsilon \}.$$

Hence, since we already know how to enumerate the set of all strings over this alphabet,
we will also be able to enumerate all regex by simply enumerating through all strings
and filtering out all strings which aren't valid regular expressions.

### Issues with this approach
This approach, while simple, does not work out well in practice. Clearly, a large
number of strings generated will not be valid regular expressions. Additionally,
it is non-trivial to validate that a generated string is or is not a valid
regular expression, which further decreases efficiency.

# Context Free Grammars for Regular Expressions
One might suspect that the language regular expressions can be recognised by
some context free grammar. In particular, regular expressions contain matching
parentheses, with specific types of subexpressions allowed within each pair of
parentheses, strongly hinting the context free nature of the language.

Before we try to construct a context free grammar which recognises all valid regular
expressions, it is worth restricting the set of regular expressions we want to match.

Clearly, many regular expressions are equivalent, for example, $$a | b \cong b | a.$$

Additionally, there are infinietly many ways of writing a regular expression
which recognises every string, for example
$$\Sigma^*, \Sigma^* a, \Sigma^* b, \Sigma^& aa, \dots$$.

Thus, we will consider a normal form of regular expression.

Consider the following context free grammar. For brevity we will let '+' denote
the regex union operation, and '|' denote that there are multiple productions
from a single variable in the grammar.

====
S → Q | A | T | C | K
Q → (A + ε) | (T + ε) | (C + ε)
A → (T + AT) | (C + AC) | (K + AK)
AT → T | (T + AT) | AC
AC → C | (C + AC) | AK
AK → K | (K + AK)
T → a1 | a2 | · · · | ak
C → (C0 C0) | (C0 C)
C0 → Q | A | T | K
K → A∗ | T∗ | C∗
====

We can express this in python as follows:

In [ ]:
S = "<S>"
Q = "<Q>"
A = "<A>"
AT = "<AT>"
AC = "<AC>"
AK = "<AK>"
T = "<T>"
C = "<C>"
C0 = "<C0>"
K = "<K>"
EPSILON = "\u03b5"

def make_cfg(alphabet: str) -> dict[str, list[list[str]]]:
    return {
        S: [[Q], [A], [T], [C], [K]],
        Q: [["(", A, "|", EPSILON, ")"], ["(", T, "|", EPSILON, ")"], ["(", C, "|", EPSILON, ")"]],
        A: [["(", T, "|", AT, ")"], ["(", C, "|", AC, ")"], ["(", K, "|", AK, ")"]],
        AT: [[T], ["(", T, "|", AT, ")"], [AC]],
        AC: [[C], ["(", C, "|", AC, ")"], [AK]],
        AK: [[K], [K, "|", AK]],
        T: [[c] for c in alphabet],
        C: [["(", C0, C0, ")"], ["(", C0, C, ")"]],
        C0: [[Q], [A], [T], [K]],
        K: [["(", A, "*", ")"], ["(", T, "*", ")"], ["(", C, "*", ")"]]
    }

Explanation:
- S -> start
- Q -> possibly empty productions
- T -> terminals
- C -> concatenations
    - C ->\* C0 C0 ... C0 (any amount of C0)
    - C0 can be anything
- A -> unions
- K -> kleene stars (can be empty so no need for Q)
- AT/AC/AK -> T/C/K for RHS of union

## Enumerating productions of a CFG
We can enumerate all the strings produced by a Context Free Grammar by essentially
enumerating through all leftmost derivations of length k in increasing order, in the same way that
we could enumerate through all strings by generating all strings of length k in
increasing order of k.

To do this, we maintain a queue, initially containing only the start state.

Then, on each iteration, we pop the first element in the queue, and find the
leftmost variable in the string of variables and terminals.

Then, we produce all possible strings that can be obtained by performing a single
production rule on that variable. If any of these strings contain no variables,
we say we generate that string. Otherwise, we push all strings containing at
least one variable to the back of the queue and repeat the process.

This can be implemented in python as follows:

In [ ]:
def generate(alphabet: str) -> Generator[str, None, None]:
    cfg = make_cfg(alphabet)
    queue: Deque[list[str]] = Deque()
    queue.append(["<S>"])

    while True:
        curr = queue.popleft()

        for i, symbol in enumerate(curr):
            if len(symbol) > 1:  # symbol is a variable
                queue.extend(
                    [
                        curr[:i] + production + curr[i + 1 :]
                        for production in cfg[symbol]
                    ]
                )
                break
        else:
            yield "".join(curr)